# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [2]:
consumer_data = pd.read_csv('/kaggle/input/financial-consumer-csv/Financial Consumer Complaints.csv')
consumer_data

,Complaint ID,Date Sumbited,Product,Sub-product,Issue,Sub-issue,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date Received,Company response to consumer,Timely response?,Consumer disputed?
0,2824926,2/24/18,Credit card or prepaid card,General-purpose credit card or charge card,Closing your account,Company closed your account,NaN,Cumulus Financial,FL,33179,NaN,Consent not provided,Web,2/24/18,Closed with explanation,Yes,NaN
1,1515991,8/12/15,Debt collection,Credit card,Improper contact or sharing of info,Contacted employer after asked not to,NaN,Cumulus Financial,CA,93561,NaN,Consent not provided,Web,8/12/15,Closed with explanation,Yes,No
2,1145261,12/5/14,Bank account or service,Checking account,Deposits and withdrawals,"""""",NaN,Cumulus Financial,CA,91792,NaN,NaN,Referral,12/8/14,Closed with explanation,Yes,No
3,2573763,7/13/17,Mortgage,Conventional home mortgage,Trouble during payment process,"""""",Company has responded to the consumer and the ...,Cumulus Financial,CA,92629,NaN,Consent not provided,Web,7/13/17,Closed with explanation,Yes,NaN
4,2573113,7/13/17,Mortgage,Conventional home mortgage,Struggling to pay mortgage,"""""",NaN,Cumulus Financial,NY,10016,NaN,Consent not provided,Web,7/13/17,Closed with explanation,Yes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75508,3080971,11/22/18,Debt collection,Credit card debt,Communication tactics,Frequent or repeated calls,Company has responded to the consumer and the ...,Cumulus Financial,GA,31701,NaN,Consent provided,Web,11/22/18,Closed with monetary relief,Yes,NaN
75509,3111624,12/28/18,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,NaN,Cumulus Financial,CA,90046,NaN,NaN,Phone,12/28/18,Closed with monetary relief,Yes,NaN
75510,3141604,2/1/19,Checking or savings account,Checking account,Managing an account,Deposits and withdrawals,Company has responded to the consumer and the ...,Cumulus Financial,MD,20874,NaN,NaN,Referral,2/4/19,Closed with explanation,Yes,NaN
75511,3081601,11/23/18,Debt collection,I do not know,Attempts to collect debt not owed,Debt was result of identity theft,Company has responded to the consumer and the ...,Cumulus Financial,GA,302XX,NaN,Consent provided,Web,11/23/18,Closed with non-monetary relief,Yes,NaN


# Wrangling Data

In [3]:
def wrangle_consumer_data(df):
    # Drop unnecessary columns
    df = df.drop(['Tags', 'Consumer consent provided?', 'ZIP code'], axis=1)
    
    # Replace spaces with underscores in column names
    df.columns = [col.replace(' ', '_') for col in df.columns]
    
    # Convert date columns to datetime
    df['Date_Sumbited'] = pd.to_datetime(df['Date_Sumbited'], format='%m/%d/%y')
    df['Date_Received'] = pd.to_datetime(df['Date_Received'], format='%m/%d/%y')
    
    # Replace empty string values in 'Sub-product' and 'Sub-issue'
    df['Sub-product'] = df['Sub-product'].replace('""', "I do not know")
    df['Sub-issue'] = df['Sub-issue'].replace('""', "I do not know")
    
    # Fill missing values in 'Company_public_response' with a default text
    df['Company_public_response'].fillna('Pending to provide a public response', inplace=True)
    
    # Find the most frequent value in the 'State' column
    most_frequent_state = df['State'].mode()[0]

    # Fill missing values with the most frequent state
    df['State'].fillna(value=most_frequent_state, inplace=True)
    
    # Drop rows where 'Sub-issue' is NaN
    df.dropna(subset=['Sub-issue'], inplace=True)
    
    # Replace the category values with Numerical to impute missing Values
    dict_ = {
    'Yes':1,
    'No': 0
    }
    df['Consumer_disputed?'] = df['Consumer_disputed?'].map(dict_)

    # Select the column to impute
    column_to_impute = 'Consumer_disputed?'
    # Create a copy of the dataset with the selected column
    impute_data = df[[column_to_impute]].copy()

    # Create an instance of the KNNImputer with the desired parameters
    imputer = KNNImputer(n_neighbors=5)

    # Fit and transform the selected column using the imputer
    impute_data[column_to_impute] = imputer.fit_transform(impute_data[[column_to_impute]])

    # Replace the original column in the dataset with the imputed values
    df[column_to_impute] = impute_data[column_to_impute]

    # Creating a dictionary for replacement
    replacement_dict = {value: 'yes' if value >= 0.5 else 'no' for value in df['Consumer_disputed?'].unique()}

    # Applying the replacement to the 'Consumer_disputed?' column
    df['Consumer_disputed?'] = df['Consumer_disputed?'].map(replacement_dict)
    
    df.drop_duplicates(inplace=True)

    return df

In [4]:
consumer_data = wrangle_consumer_data(consumer_data)

In [5]:
consumer_data.head()

,Complaint_ID,Date_Sumbited,Product,Sub-product,Issue,Sub-issue,Company_public_response,Company,State,Submitted_via,Date_Received,Company_response_to_consumer,Timely_response?,Consumer_disputed?
0,2824926,2018-02-24,Credit card or prepaid card,General-purpose credit card or charge card,Closing your account,Company closed your account,Pending to provide a public response,Cumulus Financial,FL,Web,2018-02-24,Closed with explanation,Yes,no
1,1515991,2015-08-12,Debt collection,Credit card,Improper contact or sharing of info,Contacted employer after asked not to,Pending to provide a public response,Cumulus Financial,CA,Web,2015-08-12,Closed with explanation,Yes,no
2,1145261,2014-12-05,Bank account or service,Checking account,Deposits and withdrawals,I do not know,Pending to provide a public response,Cumulus Financial,CA,Referral,2014-12-08,Closed with explanation,Yes,no
3,2573763,2017-07-13,Mortgage,Conventional home mortgage,Trouble during payment process,I do not know,Company has responded to the consumer and the ...,Cumulus Financial,CA,Web,2017-07-13,Closed with explanation,Yes,no
4,2573113,2017-07-13,Mortgage,Conventional home mortgage,Struggling to pay mortgage,I do not know,Pending to provide a public response,Cumulus Financial,NY,Web,2017-07-13,Closed with explanation,Yes,no


In [6]:
consumer_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74762 entries, 0 to 75512
Data columns (total 14 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   Complaint_ID                  74762 non-null  int64         
 1   Date_Sumbited                 74762 non-null  datetime64[ns]
 2   Product                       74762 non-null  object        
 3   Sub-product                   74762 non-null  object        
 4   Issue                         74762 non-null  object        
 5   Sub-issue                     74762 non-null  object        
 6   Company_public_response       74762 non-null  object        
 7   Company                       74762 non-null  object        
 8   State                         74762 non-null  object        
 9   Submitted_via                 74762 non-null  object        
 10  Date_Received                 74762 non-null  datetime64[ns]
 11  Company_response_to_consumer  747

In [7]:
# Save the Datast to use in Power BI Dashboard
consumer_data.to_csv('Financial_Consumer_data.csv')